In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# http://pytorch.org/

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision
import numpy as np

In [0]:
dirc = '/content/gdrive/My Drive/Colab Notebooks/flower_data'
train_dir = dirc + '/train'
valid_dir = dirc + '/valid'

train_transform = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
valid_transform = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(train_dir, transform = train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transform)

In [0]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)

In [0]:
import json

with open('/content/gdrive/My Drive/Colab Notebooks/flower_data/pytorch_challenge-master/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [8]:
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

In [0]:
#DISPLAY Image
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
def im_convert(tensor):
    """ Display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image
  
fig, (ax1) = plt.subplots(1, figsize=(10, 10))
# content and style ims side-by-side
ax1.imshow(im_convert(images[11]))


In [0]:
images, labels = next(iter(trainloader))

In [9]:
model = models.densenet121(pretrained=True)
#model.aux_logits=False
model

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [0]:
!pip install -q collections

In [11]:
model.classifier.out_features

1000

In [0]:
n_inputs = model.classifier.in_features
n_outputs = 102
last_layer = nn.Linear(n_inputs, n_outputs)
model.classifier = last_layer

In [13]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
for param in model.classifier.parameters():
  param.requires_grad = True

In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [0]:
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
  model.cuda()

In [21]:
#TRAINING 


n_epochs = 30
valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):

    
    train_loss = 0.0
    valid_loss = 0.0
    
    
    model.train()
    for data, target in trainloader:
        
        if train_on_gpu:
          data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()        
        output = model(data)        
        loss = criterion(output, target)        
        loss.backward()        
        optimizer.step()       
        train_loss += loss.item()*data.size(0)
        
   
    model.eval()
    for data, target in testloader:
        
        
        if train_on_gpu:
          data, target = data.cuda(), target.cuda()
        
        output = model(data)        
        loss = criterion(output, target)         
        valid_loss += loss.item()*data.size(0)
    
    
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(testloader.dataset)
        
   
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
   
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 2.776470 	Validation Loss: 1.287040
Validation loss decreased (inf --> 1.287040).  Saving model ...
Epoch: 2 	Training Loss: 1.238198 	Validation Loss: 0.690080
Validation loss decreased (1.287040 --> 0.690080).  Saving model ...
Epoch: 3 	Training Loss: 0.862498 	Validation Loss: 0.456561
Validation loss decreased (0.690080 --> 0.456561).  Saving model ...
Epoch: 4 	Training Loss: 0.663096 	Validation Loss: 0.354843
Validation loss decreased (0.456561 --> 0.354843).  Saving model ...
Epoch: 5 	Training Loss: 0.592547 	Validation Loss: 0.315026
Validation loss decreased (0.354843 --> 0.315026).  Saving model ...
Epoch: 6 	Training Loss: 0.506224 	Validation Loss: 0.278615
Validation loss decreased (0.315026 --> 0.278615).  Saving model ...
Epoch: 7 	Training Loss: 0.463086 	Validation Loss: 0.258323
Validation loss decreased (0.278615 --> 0.258323).  Saving model ...
Epoch: 8 	Training Loss: 0.446310 	Validation Loss: 0.256241
Validation loss decreased (0.25832

In [0]:
from google.colab import files
files.download('model_cifar.pt')

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 142.5 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
train_on_gpu

True

In [0]:
images, labels = next(iter(trainloader))
model.cpu()
#images.cuda()
#labels.cuda()
output = model(images)
print(output)

In [41]:
test = models.densenet121(pretrained = False)
test.classifier.in_features

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


1024

In [0]:
classify = nn.Linear(test.classifier.in_features, 102)
test.classifier = classify

In [34]:
test

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
state_dict = torch.load('model_cifar.pt')
test.load_state_dict(state_dict)

In [36]:
test

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
for param in test.parameters():
  param.requires_grad = False

In [0]:
rl = 0.0
if train_on_gpu:
  test.cuda()
test.eval()
for data, labels in trainloader:
  
  data, labels = data.cuda(), labels.cuda()
  output = test(data)
  
  _, pred = torch.max(output, 1)
  rl += torch.sum(pred == labels.data)


  

In [53]:
acc = rl.double()/len(testloader.dataset)
print(acc)

tensor(0.9560, device='cuda:0', dtype=torch.float64)


In [51]:
torch.version.cuda

'9.2.148'